In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from msslib.utils import *

import random

from pystruct.models import GridCRF
import pystruct.learners as ssvm
from pystruct.utils import SaveLogger

from sklearn.decomposition import PCA

import datetime
import random

In [3]:
test_runs = ["A","B","C","D","E","F"]

In [4]:
unwanted_recto = ["VA223RN","VA327RN","VA001RN","VA002RN","VA006RN","VA007RN","VA008RN","VA009RN"]

def get_set_paths(directory):
    return sorted(list(filter(lambda x: "RN" in x and not any(s in x for s in unwanted_recto) , listpaths(directory))))

In [5]:
label_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/10blocks/labels/"
data_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/10blocks/data/"    

label_paths = get_set_paths(label_dir)
data_paths = get_set_paths(data_dir)

path_pairs = list(zip(label_paths, data_paths))
print(len(path_pairs))

150


In [7]:
for letter in test_runs:
    split_at = 20
    random.shuffle(path_pairs)
    train = path_pairs[:split_at]
    test = path_pairs[split_at:]
    
    oot_path = os.path.join("/home/finlay/HMT/data/VenetusA/1200Hx900W/10blocks/PCA_REAL_20_Grid_CRF", letter)
    mkdir(oot_path)
 
    crf = GridCRF(neighborhood=4)
    clf = ssvm.OneSlackSSVM(model=crf,n_jobs=-1, C=100, inference_cache=100, tol=.1,
                           logger=SaveLogger(oot_path+'.pickle', save_every=100))
    
    pca = PCA(n_components=20)
    print(datetime.datetime.now())
    print("Training PCA %s" %letter)
    for l, d in train:
        X = flatten_vector_matrix(np.load(d))
        #y = np.squeeze(flatten_vector_matrix(np.load(l)))
        pca.fit(X)
  
    print("Training %s" %letter)
    train_X = []
    train_y = []
    for l, d in train:
        print("Training on %s" %l)
        pre_X = pca.transform(flatten_vector_matrix(np.load(d)))
        train_X.append(pre_X.reshape(120,90,20))
        train_y.append(np.squeeze(np.load(l)).astype('int'))
    print(np.array(train_X).shape)
    clf.fit(train_X, train_y)
        
    print("Testing %s" %letter)
    test_results = []
    for l, d in test:
        pre_X = pca.transform(np.load(d))
        X = pre_X.reshape((120,90, pre_X.shape[1]))
        pred = np.asarray(clf.predict(X))
        test_results.append({
            'name': only_basename(l),
            'pred': pred.reshape(120,90),
            })
    
    print("Saving %s" %letter)
    for result in test_results:
        path = format_path(oot_path, 'npy', result['name'])
        np.save(path, scale_matrix(result['pred'], 10))
    print(datetime.datetime.now())


2016-06-03 18:57:56.878831
Training PCA A


KeyboardInterrupt: 

In [18]:
clf.model.joint_feature(X[0],y[0])

IndexError: index 90 is out of bounds for axis 0 with size 90

In [14]:
X.shape

(120, 90, 20)

In [15]:
y.shape

(120, 90)